Compte tenu des spécificités de votre problème—données déséquilibrées, coûts de mauvaise classification asymétriques, et un mélange de caractéristiques catégorielles et numériques avec une faible corrélation—je recommande d'utiliser les **Machines à Boosting de Gradient**, spécifiquement **XGBoost** ou **LightGBM**.

**Arguments pour ce choix**:

- **Performance**: Les GBM surpassent souvent d'autres algorithmes dans les problèmes de données structurées.
- **Gestion des données déséquilibrées**: Méthodes intégrées pour gérer le déséquilibre des classes.
- **Fonctions de perte personnalisées**: Permettent d'incorporer le ratio de coût 10:1 pour les faux négatifs.
- **Importance des caractéristiques**: Fournissent des métriques d'importance des caractéristiques.
- **Efficacité**: Efficaces avec de grands ensembles de données.
- **Gestion des types de caractéristiques**: Gèrent efficacement les caractéristiques catégorielles et numériques.
- **Faible corrélation des caractéristiques**: Moins de problèmes liés à la multicolinéarité.
- **Techniques d'atténuation**: Arrêt précoce pour éviter le surapprentissage et les valeurs SHAP ou LIME pour une meilleure interprétabilité.

# Validate model selection after first feature selection

## Load libraries

In [32]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, auc, confusion_matrix, f1_score

import pandas as pd
import numpy as np

import re

## Set global parameters

In [2]:
data_path = 'C:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/data/03_primary/df_agg.csv'
test_size = 0.2
random_state = 18
cost_fn = 10
cost_fp = 1

## Load data

In [3]:
# Load data
raw_data = pd.read_csv(data_path)

C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8288\2395300263.py:2: DtypeWarning: Columns (756,757,761,762,766,767,771,772,776,777,781,782,786,787,791,792) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(data_path)


In [4]:
data = raw_data

In [35]:
data.columns = [re.sub('[^A-Za-z0-9_]+', '', col) for col in data.columns]

In [5]:
data.dtypes.value_counts()

float64    606
bool       133
int64       42
object      16
Name: count, dtype: int64

In [6]:
# Fonction pour afficher les colonnes de type objet avec plus de 2 valeurs uniques
def display_unique_values(df):
    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        unique_values = df[col].nunique()
        if unique_values > 2:
            print(f"Colonne: {col}, Valeurs uniques: {df[col].unique()}")

display_unique_values(data)


In [7]:
# Fonction pour convertir les colonnes objet avec 1 à 2 valeurs uniques en booléen ou int
def convert_object_columns(df):
    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        # Remplacer les NaN par la valeur la plus représentée
        most_frequent_value = df[col].mode()[0]
        df[col] = df[col].fillna(most_frequent_value)

        unique_values = df[col].nunique()
        if unique_values == 1:
            df[col] = df[col].astype('bool')
        elif unique_values == 2:
            df[col] = df[col].astype('category').cat.codes.astype('int8')
    return df

# Convertir les colonnes objet avec 1 à 2 valeurs uniques en booléen ou int
data = convert_object_columns(data)


C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8288\3074807741.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(most_frequent_value)
C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8288\3074807741.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(most_frequent_value)
C:\Users\Z478SG\AppData\Local\Temp\ipykernel_8288\3074807741.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To 

In [8]:
# Fonction pour déterminer le type de données approprié
def determine_int_type(max_value):
    if max_value <= np.iinfo(np.int8).max:
        return np.int8
    elif max_value <= np.iinfo(np.int16).max:
        return np.int16
    elif max_value <= np.iinfo(np.int32).max:
        return np.int32
    else:
        return np.int64

# Parcourir chaque colonne et convertir le type de données si nécessaire
for col in data.select_dtypes(include=[np.int64]).columns:
    max_value = data[col].max()
    new_type = determine_int_type(max_value)
    data[col] = data[col].astype(new_type)

In [9]:
data.dtypes.value_counts()

float64    606
bool       141
int8        49
int32        1
Name: count, dtype: int64

In [10]:
# Remove lines with TARGET = NaN
data = data.dropna(subset=["TARGET"])

In [45]:
X = data.drop("TARGET", axis=1)
y = data["TARGET"]

In [36]:
## TEST
# Séparer les lignes avec TARGET égal à 0 et celles avec TARGET égal à 1
df_0 = data[data['TARGET'] == 0]
df_1 = data[data['TARGET'] == 1]

# Calculer le nombre de lignes nécessaires pour chaque catégorie
n_0 = int(1000 * 0.92)  # 92% de 1000
n_1 = 1000 - n_0        # 8% de 1000

# Sélectionner aléatoirement le nombre approprié de lignes pour chaque catégorie
df_0_sampled = df_0.sample(n=n_0, random_state=42)
df_1_sampled = df_1.sample(n=n_1, random_state=42)

# Concaténer les lignes sélectionnées pour obtenir le DataFrame final
df_sampled = pd.concat([df_0_sampled, df_1_sampled])

# Mélanger les lignes pour obtenir un DataFrame final aléatoire
df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

X = df_sampled.drop("TARGET", axis=1)
y = df_sampled["TARGET"]

In [37]:
# Print columns types and number of columns in each type
X.dtypes.value_counts()


float64    605
bool       141
int8        49
int32        1
Name: count, dtype: int64

### Process object columns

In [13]:
# Print columns name that are of type object
X.select_dtypes(include='object').columns

Index([], dtype='object')

In [8]:
# Print columns content that are of type object
X.select_dtypes(include='object').head()

""
0
1
2
3
4


In [15]:
# Try to convert object columns to bool
if len(X.select_dtypes(include='object').columns)!=0:
    X = pd.get_dummies(X, drop_first=True)

In [16]:
# Print columns types and number of columns in each type
X.dtypes.value_counts()

float64    605
bool       141
int8        49
int32        1
Name: count, dtype: int64

### Object columns processed well!

### Process infinit values

In [17]:
inf_cols_mask = np.isinf(X).any()

In [18]:
# Print column names with infinite values
inf_cols = X.columns.to_series()[inf_cols_mask].tolist()
inf_cols

[]

In [19]:
# Print how much rows have infinite values
inf_rows_mask = np.isinf(X).any(axis=1)
len(X[inf_rows_mask])

0

In [20]:
# Print the rows that have infinite values
X[inf_rows_mask][inf_cols]

""


In [38]:
# Print y values on rows where X have inf values
y[inf_rows_mask]

Series([], Name: TARGET, dtype: float64)

In [21]:
for col in inf_cols:
    print(X[col].describe())

In [22]:
# Replace inf values with max
for col in inf_cols:
    if col in X.columns:  # Check if the column exists in the DataFrame
        max_value = X[col][X[col] != np.inf].max()  # Get the max value excluding inf
        X[col] = X[col].replace([np.inf, -np.inf], max_value)  # Replace inf values

In [24]:
inf_cols_mask = np.isinf(X).any()
# Print column names with infinite values
inf_cols = X.columns.to_series()[inf_cols_mask].tolist()
inf_cols

[]

### Infinite values processed!

## Split data

In [39]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

## Create model

### Select a model

Models to test:
- RandomForestClassifier
- XGBoost
- LightGBM
- Logistic Regression avec pondération des classes

In [26]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [40]:
# Random forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=random_state,)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Mean CV AUC-ROC: {np.mean(cv_scores)}, Std CV AUC-ROC: {np.std(cv_scores)}")

# Entraîner le modèle sur l'ensemble d'entraînement complet
model.fit(X_train, y_train)

# Prédire les probabilités sur l'ensemble de test
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")


Mean CV AUC-ROC: 0.5959485093413666, Std CV AUC-ROC: 0.08054735453519436
Test AUC-ROC: 0.6257633032858388


In [41]:
# XGBoost classifier for binary classification outputing probabilities
model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Mean CV AUC-ROC: {np.mean(cv_scores)}, Std CV AUC-ROC: {np.std(cv_scores)}")

# Entraîner le modèle sur l'ensemble d'entraînement complet
model.fit(X_train, y_train)

# Prédire les probabilités sur l'ensemble de test
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")

Mean CV AUC-ROC: 0.6785258178115321, Std CV AUC-ROC: 0.052833399639566604
Test AUC-ROC: 0.5953765629543472


In [42]:
# XGBoost classifier for binary classification outputing scores
model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:hinge')
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Mean CV AUC-ROC: {np.mean(cv_scores)}, Std CV AUC-ROC: {np.std(cv_scores)}")

# Entraîner le modèle sur l'ensemble d'entraînement complet
model.fit(X_train, y_train)

# Prédire les probabilités sur l'ensemble de test
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")

Mean CV AUC-ROC: 0.5279742458313887, Std CV AUC-ROC: 0.02399696604252801
Test AUC-ROC: 0.5388194242512357


In [43]:
# LightGBM classifier
model = LGBMClassifier(n_estimators=100, random_state=random_state)
X_train.columns = [re.sub('[^A-Za-z0-9_]+', '', col) for col in X_train.columns]
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Mean CV AUC-ROC: {np.mean(cv_scores)}, Std CV AUC-ROC: {np.std(cv_scores)}")

# Entraîner le modèle sur l'ensemble d'entraînement complet
model.fit(X_train, y_train)

# Prédire les probabilités sur l'ensemble de test
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")

[LightGBM] [Info] Number of positive: 48, number of negative: 592
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013901 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23321
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 548
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.075000 -> initscore=-2.512306
[LightGBM] [Info] Start training from score -2.512306
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [78]:
# Supposons que X est votre DataFrame
missing_percentage = X.isnull().mean() * 100
missing_table = pd.DataFrame({'Column': X.columns, 'Missing Percentage': missing_percentage})

# Trier par ordre croissant du pourcentage de valeurs manquantes
missing_table = missing_table.sort_values(by='Missing Percentage', ascending=True)

# Afficher le tableau trié
print("Tableau trié par ordre croissant du pourcentage de valeurs manquantes:")
missing_table

Tableau trié par ordre croissant du pourcentage de valeurs manquantes:


,Column,Missing Percentage
SK_ID_CURR,SK_ID_CURR,0.0
OCCUPATION_TYPE_Salesstaff,OCCUPATION_TYPE_Salesstaff,0.0
OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Secretaries,0.0
OCCUPATION_TYPE_Securitystaff,OCCUPATION_TYPE_Securitystaff,0.0
OCCUPATION_TYPE_Waitersbarmenstaff,OCCUPATION_TYPE_Waitersbarmenstaff,0.0
...,...,...
CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,99.3
CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,99.3
CC_AMT_DRAWINGS_POS_CURRENT_MIN,CC_AMT_DRAWINGS_POS_CURRENT_MIN,99.3
CC_CNT_DRAWINGS_OTHER_CURRENT_VAR,CC_CNT_DRAWINGS_OTHER_CURRENT_VAR,99.3


In [80]:

# Compter le nombre de colonnes avec différents pourcentages de valeurs manquantes
missing_counts = {
    '>100%': (missing_table['Missing Percentage'] == 100).sum(),
    '90-100%': ((missing_table['Missing Percentage'] >= 90) & (missing_table['Missing Percentage'] < 100)).sum(),
    '75-90%': ((missing_table['Missing Percentage'] >= 75) & (missing_table['Missing Percentage'] < 90)).sum(),
    '50-75%': ((missing_table['Missing Percentage'] >= 50) & (missing_table['Missing Percentage'] < 75)).sum(),
    '25-50%': ((missing_table['Missing Percentage'] >= 25) & (missing_table['Missing Percentage'] < 50)).sum(),
    '<25%': (missing_table['Missing Percentage'] < 25).sum()
}

# Afficher le nombre de colonnes avec différents pourcentages de valeurs manquantes
print("\nNombre de colonnes avec différents pourcentages de valeurs manquantes:")
for key, value in missing_counts.items():
    print(f"{key}: {value} colonnes")



Nombre de colonnes avec différents pourcentages de valeurs manquantes:
>100%: 0 colonnes
90-100%: 131 colonnes
75-90%: 36 colonnes
50-75%: 100 colonnes
25-50%: 312 colonnes
<25%: 217 colonnes


In [85]:
# Filtrer les colonnes qui ont plus de 90% de valeurs manquantes
columns_to_drop = missing_table[missing_table['Missing Percentage'] > 90]['Column']

# Supprimer ces colonnes du DataFrame
X_less_na = X.drop(columns=columns_to_drop)

In [86]:
# Supposons que X est votre DataFrame
missing_percentage = X_less_na.isnull().mean() * 100
missing_table = pd.DataFrame({'Column': X_less_na.columns, 'Missing Percentage': missing_percentage})

# Trier par ordre croissant du pourcentage de valeurs manquantes
missing_table = missing_table.sort_values(by='Missing Percentage', ascending=True)

# Afficher le tableau trié
print("Tableau trié par ordre croissant du pourcentage de valeurs manquantes:")
missing_table

Tableau trié par ordre croissant du pourcentage de valeurs manquantes:


,Column,Missing Percentage
SK_ID_CURR,SK_ID_CURR,0.0
WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,0.0
ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Advertising,0.0
ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Agriculture,0.0
ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Bank,0.0
...,...,...
CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MAX,81.9
CLOSED_AMT_ANNUITY_MEAN,CLOSED_AMT_ANNUITY_MEAN,81.9
ACTIVE_AMT_ANNUITY_MEAN,ACTIVE_AMT_ANNUITY_MEAN,82.0
ACTIVE_AMT_ANNUITY_MAX,ACTIVE_AMT_ANNUITY_MAX,82.0


In [87]:
# Calculer la moyenne de chaque colonne
means = X_less_na.mean()

# Remplacer les valeurs manquantes par la moyenne de chaque colonne
X_no_na = X_less_na.fillna(means)


In [89]:
# Supposons que X est votre DataFrame
missing_percentage = X_no_na.isnull().mean() * 100
missing_table = pd.DataFrame({'Column': X_no_na.columns, 'Missing Percentage': missing_percentage})

# Trier par ordre croissant du pourcentage de valeurs manquantes
missing_table = missing_table.sort_values(by='Missing Percentage', ascending=True)

# Afficher le tableau trié
print("Tableau trié par ordre croissant du pourcentage de valeurs manquantes:")
missing_table

Tableau trié par ordre croissant du pourcentage de valeurs manquantes:


,Column,Missing Percentage
SK_ID_CURR,SK_ID_CURR,0.0
PREV_NAME_CONTRACT_STATUS_nan_MEAN,PREV_NAME_CONTRACT_STATUS_nan_MEAN,0.0
PREV_NAME_PAYMENT_TYPE_Cashthroughthebank_MEAN,PREV_NAME_PAYMENT_TYPE_Cashthroughthebank_MEAN,0.0
PREV_NAME_PAYMENT_TYPE_Cashlessfromtheaccountoftheemployer_MEAN,PREV_NAME_PAYMENT_TYPE_Cashlessfromtheaccounto...,0.0
PREV_NAME_PAYMENT_TYPE_Noncashfromyouraccount_MEAN,PREV_NAME_PAYMENT_TYPE_Noncashfromyouraccount_...,0.0
...,...,...
ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_University,0.0
ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_XNA,0.0
FONDKAPREMONT_MODE_notspecified,FONDKAPREMONT_MODE_notspecified,0.0
FONDKAPREMONT_MODE_regoperaccount,FONDKAPREMONT_MODE_regoperaccount,0.0


In [90]:
X_no_na_train, X_no_na_test, y_train, y_test = train_test_split(X_no_na, y, test_size=test_size, random_state=random_state)

In [92]:
# Logistic regression classifier with class weight
model = LogisticRegression(class_weight={0: cost_fn, 1: cost_fp}, random_state=random_state, max_iter=1000)
cv_scores = cross_val_score(model, X_no_na_train, y_train, cv=5, scoring='roc_auc')
print(f"Mean CV AUC-ROC: {np.mean(cv_scores)}, Std CV AUC-ROC: {np.std(cv_scores)}")

# Entraîner le modèle sur l'ensemble d'entraînement complet
model.fit(X_no_na_train, y_train)

# Prédire les probabilités sur l'ensemble de test
y_pred_proba = model.predict_proba(X_no_na_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Mean CV AUC-ROC: 0.5439901636330207, Std CV AUC-ROC: 0.05880601867420225
Test AUC-ROC: 0.523407967432393


C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On choisit bien LGBMClassifier qui a le meilleur AUC-ROC : 0.75